For input to NN:
Only MFCCs are extracted over 3 second segmented of the audio clip(1 song = 10 parts of 3 seconds each)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv



In [ ]:
song_dir= '/content/drive/My Drive/RP/genres/'

In [ ]:
def get_features(y, sr):
  features={}
  # MFCC 
  mfcc = librosa.feature.mfcc(y, sr=sr, n_mfcc=13)
  for idx, v_mfcc in enumerate(mfcc):
    features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
  # cal statistical moments
  def get_moments(descriptors):
    result = {}
    for k, v in descriptors.items():
      result['{}_max'.format(k)] = np.max(v)
      result['{}_min'.format(k)] = np.min(v)
      result['{}_mean'.format(k)] = np.mean(v)
      result['{}_std'.format(k)] = np.std(v)
      result['{}_kurtosis'.format(k)] = kurtosis(v)
      result['{}_skew'.format(k)] = skew(v)
    return result
    
  dict_agg_features = get_moments(features)
    
  return dict_agg_features

In [ ]:
num_segments=10  #vary acc to req. eg: 5,10 
sample_rate = 22050
track_duration = 30 
samples_per_track = sample_rate * track_duration
samples_per_segment = int(samples_per_track / num_segments)

def final_extraction(song_dir):
  genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
  last_features=[]
  for genre in genres:
    path_genre = song_dir + '/' + genre 
    for song in os.listdir(path_genre):        
      path_song = path_genre + '/' + song
      y, sr = librosa.load(path_song,sr=sample_rate)
      for d in range(num_segments):
        # calculate start and finish sample for current segment
        start = samples_per_segment * d #d=0, start=0
        finish = start + samples_per_segment # 0+5=5
        features = get_features(y[start:finish], sr)
        features['genre'] = genre
        last_features.append(features)
  return pd.DataFrame(last_features)

In [ ]:
%%time

df_features = final_extraction(song_dir)
df_features.to_csv('mfcc.csv', index=False)


CPU times: user 3min 16s, sys: 2min 35s, total: 5min 51s
Wall time: 3min 3s


In [ ]:
df_features = pd.read_csv('mfcc.csv')

In [ ]:
df_features.shape

In [ ]:
df_features.head()

,mfcc_0_max,mfcc_0_min,mfcc_0_mean,mfcc_0_std,mfcc_0_kurtosis,mfcc_0_skew,mfcc_1_max,mfcc_1_min,mfcc_1_mean,mfcc_1_std,mfcc_1_kurtosis,mfcc_1_skew,mfcc_2_max,mfcc_2_min,mfcc_2_mean,mfcc_2_std,mfcc_2_kurtosis,mfcc_2_skew,mfcc_3_max,mfcc_3_min,mfcc_3_mean,mfcc_3_std,mfcc_3_kurtosis,mfcc_3_skew,mfcc_4_max,mfcc_4_min,mfcc_4_mean,mfcc_4_std,mfcc_4_kurtosis,mfcc_4_skew,mfcc_5_max,mfcc_5_min,mfcc_5_mean,mfcc_5_std,mfcc_5_kurtosis,mfcc_5_skew,mfcc_6_max,mfcc_6_min,mfcc_6_mean,mfcc_6_std,mfcc_6_kurtosis,mfcc_6_skew,mfcc_7_max,mfcc_7_min,mfcc_7_mean,mfcc_7_std,mfcc_7_kurtosis,mfcc_7_skew,mfcc_8_max,mfcc_8_min,mfcc_8_mean,mfcc_8_std,mfcc_8_kurtosis,mfcc_8_skew,mfcc_9_max,mfcc_9_min,mfcc_9_mean,mfcc_9_std,mfcc_9_kurtosis,mfcc_9_skew,mfcc_10_max,mfcc_10_min,mfcc_10_mean,mfcc_10_std,mfcc_10_kurtosis,mfcc_10_skew,mfcc_11_max,mfcc_11_min,mfcc_11_mean,mfcc_11_std,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,genre
0,-6.147503,-315.573669,-160.181381,70.142807,-0.520162,-0.053890,176.595673,-11.409155,119.630424,39.994968,-0.165291,-0.751660,8.235102,-81.617920,-32.245754,17.397913,-0.281810,0.227252,86.173431,-3.133601,24.449011,15.865477,1.470297,0.950947,0.318795,-68.568909,-32.649342,16.664333,-0.693737,-0.241460,42.184769,-13.885689,13.284251,12.807674,-0.681826,0.317151,0.274350,-60.352791,-22.984686,10.742055,0.949779,-0.975547,26.916073,-27.954872,0.905412,11.579039,-0.609163,0.344272,22.398947,-44.258591,-13.604530,13.023029,-0.223690,0.063000,30.684137,-19.783592,2.381121,11.138239,-0.668489,0.339284,24.979959,-36.597332,-15.892393,8.429502,4.628271,1.132685,24.054575,-25.544872,-3.992007,8.606446,0.100298,0.361005,18.835262,-23.745459,-4.833901,10.294226,-0.798276,0.278480,blues
1,-17.793070,-350.602875,-148.886612,71.266716,-0.327152,-0.542607,177.832031,57.217682,119.406181,29.670467,-0.925985,0.025420,19.228203,-79.058418,-38.778679,17.812393,0.042350,0.597614,62.289505,-11.598762,26.627285,14.336739,-0.306717,-0.034557,-0.092132,-80.042130,-42.240627,18.192192,-0.888200,-0.004776,49.037830,-15.620793,14.962059,11.940646,-0.483307,-0.103732,-4.980200,-67.112244,-25.051838,9.697455,2.066697,-0.947084,35.566586,-15.166626,8.034304,11.547892,-0.958414,0.244911,22.373924,-43.282532,-15.917373,13.580447,-0.488991,0.124393,30.655251,-10.113426,7.389182,9.646088,-0.996889,0.160924,16.464054,-38.248848,-17.092546,9.388429,1.357034,0.653570,35.863750,-22.127949,-2.557645,8.709692,1.320397,0.763025,19.743805,-37.547760,-8.219938,9.969503,0.344203,-0.211692,blues
2,-37.758907,-353.501129,-159.218124,70.818382,-0.024230,-0.721412,176.448212,55.123913,127.310478,23.125025,-0.415008,-0.360723,10.436086,-85.979156,-44.985493,19.241497,-0.369592,0.402934,60.916531,0.146579,28.247896,11.652804,-0.093760,0.076632,12.992712,-93.966614,-35.065773,16.167730,1.880795,-0.211646,49.880829,-13.175097,16.588394,10.771525,-0.011258,0.232406,-5.671995,-50.869164,-27.049433,8.967988,-0.408302,-0.339952,36.207348,-19.788561,3.732071,10.055588,0.731473,0.623490,12.773608,-43.256443,-14.107492,9.871629,0.027560,-0.341220,29.811180,-19.366587,3.472952,8.265056,0.402948,0.163359,20.518749,-37.100212,-14.993042,8.892298,1.399053,0.441806,34.176933,-26.468193,2.792713,11.179812,-0.170329,-0.050669,21.034733,-34.293995,-10.131114,9.998880,0.296466,0.294695,blues
3,-22.790054,-362.560791,-182.391525,81.765808,-0.832404,0.004257,194.498962,74.681747,142.233795,23.020880,0.160544,-0.375933,2.065183,-73.904480,-36.764919,16.689089,-0.677332,-0.075242,50.117062,-4.001195,23.232386,11.853209,-0.693426,-0.072148,0.266670,-100.433960,-32.181301,20.014469,1.146455,-1.186497,38.377411,-9.882708,12.990951,10.062401,-0.503992,0.122457,-5.904160,-51.450283,-28.718052,9.217214,-0.240549,-0.020828,30.412960,-19.675873,3.146388,10.778200,-0.256142,0.418698,5.713670,-40.961121,-13.716473,8.808634,0.180722,-0.403179,23.568348,-17.220497,2.665599,8.806189,-0.630623,-0.045293,2.284895,-37.734016,-15.466902,7.417944,-0.040219